### Load source code

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd
import source.players.bayesian_approximator

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import shutil
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [29]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Set batch path, batch folder need not to already exist

In [96]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/FRL" #insert folder where batch is
batch_name = "frl_vs_suqr_2" #insert batch name
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Parsing the csv to generate a batch containing the configurations and running them

In [97]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [98]:
if os.path.isdir(experiments_folder + exp_subfolder + "/" + batch_name):
    shutil.rmtree(experiments_folder + exp_subfolder + "/" + batch_name)
b.parse_batch()

In [99]:
batches = []
batches.append(b)

In [100]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=100)

### Collecting results of the runs

In [101]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, ylabel="$exp\_reg(t)$", xlabel="$t$", save=True, show=False, semilog=False,
                            path=b.results_folder_path,
                            title=confs[0].game.players[0].__class__.name + " vs " + confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the configuration "step-by-step"

In [13]:
gm = deepcopy(batches[0].configurations[0].game)

In [14]:
e = runner.Experiment(gm)

In [15]:
for i in range(50):
    #e.run_interaction_with_observations()
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

NameError: name 'util' is not defined

In [34]:
pl=gm.players[0]

In [35]:
pl.game.profiles

[<SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <FrequentistUnknownStochasticAttacker id:1 resources:1>,
 <StackelbergAttacker id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>,
 <SUQR id:1 resources:1>]